In [6]:
#!pip install datasets
# pip install accelerate
from huggingface_hub import login         #1-Authentifiziert mich bei Hugging Face
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from datasets import load_dataset
import torch

login() #2-Authentifiziert mich bei Hugging Face

##Vorbereitung für mich startet hier:
def check_gpu():                            #Checkt ob die GPU aktiv ist
   if torch.cuda.is_available():
       print("GPU is available")
       print(torch.cuda.get_device_name(0))
   else:
       print("GPU is not available")
check_gpu()


# Model und Tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/gemma-1.1-2b-it")   #Lädt das Model und Tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-1.1-2b-it",   #Model-typ (gemma instruction tuned)
    device_map="auto",          #Nutzt GPU automatisch
    torch_dtype=torch.bfloat16 ##entfernt aufgrund von Fehler (versucht Meta Tensor zu erschaffen ohne Daten)  #Spart Speicher
)
model = model.to("cuda") #Lädt Model auf die GPU, spezifisch Cuda
## Lora Einrichten
lora_config = LoraConfig(
    r=8,                          # LoRA-Rank = Anzahl der Low-Rank-Dimensionen (je kleiner, desto effizienter)
    lora_alpha=16,                #Skalierungsfaktor der LoRA-Gewichte – wie stark beeinflusst LoRA das Modell
    lora_dropout=0.1,             #Dropout auf LoRA-Layer -> hilft bei Regularisierung
    bias="none",                  #Trainiere keine Bias-Parameter (nur LoRA)
    task_type=TaskType.CAUSAL_LM, #Typ der Aufgabe -> wichtig, damit LoRA die richtigen Layer patcht (hier: Textgenerierung)
)

model = get_peft_model(model, lora_config) #Umwandlung in Fine-Tunebares Model
model.print_trainable_parameters()        #Kontrolliert ob nur die LoRA-Schichten trainieren

## Daten werden vorbereitet und geladen
dataset = load_dataset("yelp_polarity", split="train[:500]")  # Aktuell noch test daten, kein Datenset vorhanden!

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized = dataset.map(tokenize, batched=True)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./gemma-lora",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    save_total_limit=1,
)
#Training definieren
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    data_collator=data_collator,
)
#Model trainieren
trainer.train()
#LoRA-Finetuning speichern
model.save_pretrained("gemma-lora-finetuned")
tokenizer.save_pretrained("gemma-lora-finetuned")


#Testet Model ohne Fine Tuning (glaube ich)
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

#Ausgabe Anzeigen
outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))



GPU is available
Tesla T4


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: m-ramackers (m-ramackers-universit-t-des-saarlandes-saarland-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,5.501800
20,4.817000
30,4.560100
40,4.288700
50,4.340300
60,3.877200
70,3.650500
80,3.670300
90,3.724600
100,3.628900


<bos>Write me a poem about Machine Learning.

In circuits deep, where data flows unseen,
A mind of metal, where algorithms reign.


# Neuer Abschnitt